## Vistual inertia scheduling

In [125]:
import andes
import os

from statistics import fmean
from andes.interop.pandapower import to_pandapower
from andes.interop.pandapower import make_GSF, build_group_table
import gurobipy as gb
import pandas as pd
import numpy as np
import logging
import torch

logger = logging.getLogger(__name__)

# from opf import dcopf
from visopf import vis2

import andes.interop.pandapower as adpp

## Main - load IEEE 39 and external parameters


### load case from andes excel

In [126]:
# get andes case from excel
dir_path = os.path.abspath('..')
case_path = '/VIS_opf/ieee39_vis_opf.xlsx'
case = dir_path + case_path
ssa = andes.load(case, no_output=True)

### load norm parameter

In [127]:
# prepare nn data for visopf
data_path = dir_path + '/VIS_opf/NN_train'

fnorm = pd.read_csv(data_path + '/fnorm.csv')
pnorm = pd.read_csv(data_path + '/pnorm.csv')
norm = {'fnorm': fnorm, 'pnorm': pnorm }


### load nn parameter

In [128]:
# frequency nadir prediction network
fw1 = pd.read_csv(data_path + '/fw1.csv', header=None)
fw2 = pd.read_csv(data_path + '/fw2.csv', header=None)

fb1 = pd.read_csv(data_path + '/fb1.csv', header=None)
fb2 = pd.read_csv(data_path + '/fb2.csv', header=None)

In [129]:
# vsg peak power prediction network
pw1 = pd.read_csv(data_path + '/pw1.csv', header=None)
pw2 = pd.read_csv(data_path + '/pw2.csv', header=None)

pb1 = pd.read_csv(data_path + '/pb1.csv', header=None)
pb2 = pd.read_csv(data_path + '/pb2.csv', header=None)

In [130]:
nn = {
        'fw1': fw1,
        'fw2': fw2,       
        'fb1': fb1,
        'fb2': fb2,
        'pw1': pw1,
        'pw2': pw2,
        'pb1': pb1,
        'pb2': pb2,
    }

### test opf model

In [131]:
ss = vis2(norm=norm, nn=nn, dpe=0.0123, rocof_lim=0.0069, nadir_lim=0.01)

In [132]:
# define typeII gen (VSG inverter)
vsg_ieee14 = ['PV_6', 'PV_7']
vsg_ieee39 = ['PV_1', 'PV_6', 'PV_8', 'PV_9']

ss.from_andes(ssa, vsg_ieee39, Sbase=1000)

Note: Control (dynamic) parameters are renormalized based on case Sbase rather then to andes base


In [133]:
ss.build()

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


In [134]:
# revise gen cost
linearcost = [
                0.5,   # PV_1 vsg
                1, 
                1.2,
                0.8, 
                0.8, 
                0.1,     # PV_6 vsg
                1, 
                0.1,   # PV_8 vsg
                0.1,   # PV_9 vsg 
                1.5    # slack
            ]
ss.cost['c1'] = linearcost
ss.update_dict()
ss.costdict

{'PV_1': {'c2': 0.0, 'c1': 0.5, 'c0': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_2': {'c2': 0.0, 'c1': 1.0, 'c0': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_3': {'c2': 0.0, 'c1': 1.2, 'c0': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_4': {'c2': 0.0, 'c1': 0.8, 'c0': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_5': {'c2': 0.0, 'c1': 0.8, 'c0': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_6': {'c2': 0.0, 'c1': 0.1, 'c0': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_7': {'c2': 0.0, 'c1': 1.0, 'c0': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_8': {'c2': 0.0, 'c1': 0.1, 'c0': 0.0, 'cru': 0.0, 'crd': 0.0},
 'PV_9': {'c2': 0.0, 'c1': 0.1, 'c0': 0.0, 'cru': 0.0, 'crd': 0.0},
 'Slack_10': {'c2': 0.0, 'c1': 1.5, 'c0': 0.0, 'cru': 0.0, 'crd': 0.0}}

In [135]:
ss.get_res()

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 1040 rows, 678 columns and 11758 nonzeros
Model fingerprint: 0x2b007a82
Variable types: 358 continuous, 320 integer (320 binary)
Coefficient statistics:
  Matrix range     [3e-08, 1e+02]
  Objective range  [1e-01, 2e+00]
  Bounds range     [1e-01, 1e+01]
  RHS range        [3e-03, 2e+03]
Found heuristic solution: objective 41.7240000
Presolve removed 722 rows and 439 columns
Presolve time: 0.01s
Presolved: 318 rows, 239 columns, 3929 nonzeros
Variable types: 153 continuous, 86 integer (86 binary)

Root relaxation: objective 2.737580e+01, 380 iterations, 0.00 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   27.375

Msys and Dsys are normlized by devise Sbase, transform to andes Sbase when do TDS


--------------------- Results -------------------
Total Cost: 27.3793
RoCof prediction: 0.00205607 ; RoCof limit: 0.0069
Nadir prediction: -0.000396481 ; Nadir limit 0.01


(        gen      Sn         pg       pru  prd
 0      PV_1  10.400   7.996298  0.003702  0.0
 1      PV_2   8.360   1.500000  0.000000  0.0
 2      PV_3   8.437   1.000000  0.000000  0.0
 3      PV_4  11.748   7.000000  0.000000  0.0
 4      PV_5  10.802   7.000000  0.000000  0.0
 5      PV_6  10.857   7.994808  0.005192  0.0
 6      PV_7  10.252   2.082590  0.000000  0.0
 7      PV_8   9.702   6.996580  0.003420  0.0
 8      PV_9  16.841  13.993723  0.006277  0.0
 9  Slack_10  11.990   3.000000  0.000000  0.0,
     gen  Mvsg  Dvsg     pg_vsg   pru_vsg  prd_vsg  pmax_vsg  pmin_vsg
 0  PV_1   2.0   2.0   7.996298  0.003702      0.0       8.0       0.1
 1  PV_6   3.0   2.3   7.994808  0.005192      0.0       8.0       0.1
 2  PV_8   1.5   3.0   6.996580  0.003420      0.0       7.0       0.1
 3  PV_9   4.0   1.0  13.993723  0.006277      0.0      14.0       0.1,
 {'Msys': 5.9822889896059035,
  'Dsys': 1.4277631946539417,
  'Rsys': 20.933442043222,
  'Fsys': 7.668503053753275})